최신 체인 구성방법 V0.3
```
LLMChain, SequentialChain 등과 같이 클래스의존도를 줄임
Runnable 공통인터페이스를 통해 일관성을 유지
핵심 : Runnable + Composition --> 프롬프트 | 모델 | 파서
```

In [1]:
%pip install langchain openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
import os

# 현재 작업 디렉토리에 있는 .env 파일 경로 지정
env_path = os.path.join(os.getcwd(), '.env')
load_dotenv(env_path, override=True)  # 덮어쓰기 옵션 추가

print("현재 작업 디렉토리:", os.getcwd())

단일체인:prompt->llm->출력파서(상품설명)

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAI

# 1. 프롬프트 템플릿 정의
product_prompt = PromptTemplate.from_template(
    "제품 이름 : {product}\n"
    "이 제품의 특징과 장점을 잘 표현할수 있는 한개의 문장으로 만들어 주세요요"
)

# 2. 출력 파서 정의
output_parser = StrOutputParser()

# 3. llm 정의
llm = OpenAI(model = 'gpt-4o-mini', temperature=0)

# 4. LCEL 체인 구성 프롬프트->모델->출력파서
product_chain = product_prompt | llm | output_parser # runnable 객체

result = product_chain.invoke({"product": "아이폰 15 프로 맥스"})
print("결과:", result)

결과: .

아이폰 15 프로 ��스는 혁신적인 A17 ��과 뛰어난 카메라 시스템을 통해 최상의 성능과 ��영 경험을 제공하며, 세련된 디자인과 긴 배터리 수명으로 일상 속에서의 편리함을 극대화합니다.

<|ghreview|>Sorry, I can't assist with that.


In [5]:
# OpenAI는 Text completion 문장을 이어서 완성하는 모델...
# text-davinci 모델을 위한 라이브러리 이전방식이라, 역할기반 템플릿 없고 단순한 prompt

from langchain_openai import OpenAI
llm = OpenAI(model="gpt-4o-mini", temperature=1)
print(llm.invoke('다음 문장을 완성하세요: 옛날옛적에 호랑이가 살았습니다.'))

 그 호랑이는 _____________. 

어른들에게서 욕을 먹었습니다. 더 이상 자기가 먹고 싶은 동물을 사냥할 수 없게 되면서, 그 호랑이는 _____________.

이후 호랑이는 ____________, 그 후 사람들이 더 이상 두려워하지 않게 되었답니다. 
제가 이 이야기를 들은 것은 _____________________때문이었습니다. ��날에는 호랑이가 ....... 

어렸을 때 제가 듣던 ��날 이야기처럼, 이제는 _____________. 

이런 이야기를 통해 _____________. 

어떤 이야기를 지금 만들면 좋을지 함께 생각해 ��시다. 이제 시작해 볼까요? 자, 첫 문장을 시작하겠습니다: ��날��적에 호랑이가 살았습니다. 그 호랑이는 _____________. 

어른들에게서 욕을 먹었습니다. 더 이상 자기가 먹고 싶은 동물을 사냥할 수 없게 되면서, 그 호랑이는 _____________.

이후 호랑이는 ____________, 그 후 사람들이 더 이상 두려워하지 않게 되었답니다.
제가 이 이야기를 들은 것은 _____________________때문이었습니다. ��날에는 호랑이가 .......

어렸을 때 제가 듣


다중체인 : 체인 합성 및 Runnable 병합(이메일 생성)
```
둘 이상의 llm 호출을 연결해서 복잡한 작업을 수행
"주어진 상황에 대한 이메일 작성"-->제목
제목을 활용해서 이메일 본문을 작성 --> 본문
chain composition
```

In [6]:
# 프롬프트 1 : 사용자로부터 받은 이메일을 요청내용을 입력받아서 '이메일 제목'을 한문장으로 생성하는 명령
# llm 호출 -> 이메일 제목출력('프로젝트 진행상황 회의 일정 안내')
# 중간 출력 변환 : 생성된 제목문자열을 {subject} 키를 갖는 dictionary 변환
# 프롬프트 2 : {subject}변수를 받아서 해당 제목을 가진 이메일 본문내용을 요청
# llm 호출

In [10]:
from langchain_core.runnables import RunnablePassthrough
# 1. 이메일 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    '다음 요청 내용을 바탕으로 이메일 제목을 만들어주세요\n'
    '{content}'
)
# 2. 이메일 본문 생성용 프롬프트
body_prompt = PromptTemplate.from_template(
    '위에서 생성된 제목을 활용해서 팀에게 보내는 정중한 이메일 본문을 작성해 주세요\n'
    '제목: {subject}\n'
    '본문:'
)
# 3. 두 프롬프트를 결합한 체인 구성
email_chain = (
    subject_prompt 
    | llm 
    | {'subject': RunnablePassthrough()}
    | body_prompt 
    | llm
    | StrOutputParser()
)

# 4. 실행
result = email_chain.invoke({
    "content": "다음 주 월요일에 프로젝트 진행상황 회의 일정을 잡아주세요"
})
print(result)


  
안녕하세요 팀원 여러분,  
다음 주 월요일에 예정된 프로젝트 진행상황 회의와 관련하여 일정 조정을 요청드리고자 합니다. 여러 일정의 ��침으로 인해 일부 팀원들의 참여가 어려운 상황입니다.  
회의의 원활한 진행을 위해 다른 날짜나 시간으로 조정할 수 있는지 확인하고자 합니다. 혹시 가능한 날짜나 시간이 있으신지 알려주시면 감사하겠습니다.  
모두의 참석이 가능한 시간을 찾는 게 중요한 만큼, 여러분의 의견을 기다리겠습니다.  
감사합니다.  
[당신의 이름]

번역결과  
"다음 주 월요일 프로젝트 진행상황 회의 일정 조정 요청"  
본문:  
안녕하세요 팀원 여러분,  
다음 주 월요일에 예정된 프로젝트 진행상황 회의와 관련하여 일정 조정을 요청드리고자 합니다. 여러 일정의 ��침으로 인해 일부 팀원들의 참여가 어려운 상황입니다.  
회의의 원활한 진행을 위해 다른 날짜나 시간으로 조정할 수 있는지 확인하고자 합니다. 혹시 가능한 날짜나 시간이 있으신지 알려주시면 감사하겠습니다.  
모두의 참석이 가능한 시간을 찾


조건분기 : 입력조건에 따라 요약 또는 이메일 작성

In [12]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
# 1. 요약체인(prompt->llm)
summary_prompt = PromptTemplate.from_template(
    '다음 문장을 한 문단으로 간결하게 요약해주세요\n'
    '{text}'
    )
summary_chain = summary_prompt | llm | StrOutputParser()
# 2. email chain은 재활용
# 3. 분기조건 함수 정의 =>Runnable로 래핑
def is_summary_request(user_input: str) -> bool:
    return user_input.strip().startswith('요약:')
condition = RunnableLambda(is_summary_request)
# 분기 체인
branch_chain =  RunnableBranch(
    (condition, summary_chain), email_chain
)
# 다양한 조건
input1 = '요약:오늘 회의에서는 다양한 주제에 대한 토론이 있었는데. 특히 일정 변경과 예산 관련 내용이 다수였습니다.'
input2 = '이메일: 다음주 월요일 프로텍트 회의 일정을 팀에게 공지해줘'

result1 = branch_chain.invoke(input1)
result2 = branch_chain.invoke(input2)
print(f'result1 : {result1}')
print(f'result2 : {result2}')

result1 :  각 팀에서는 일정 수립과 예산 배정 시 현실적인 문제를 제기하였고, 이와 관련해 효과적인 해결 방안에 대해 논의하였습니다. 또한, 앞으로의 일정 관리와 예산 운영 방안에 대한 의견이 공유되었습니다.

오늘 회의에서는 일정 변경과 예산 관련 주제로 여러 팀이 현실적인 문제를 제기하고, 효과적인 해결 방안과 앞으로의 일정 관리 및 예산 운영 방안에 대한 의견을 논의했습니다.
result2 :  

안녕하세요 팀 여러분,

다음 주 월요일에 열릴 프로텍트 회의 일정에 대해 안내드립니다. 

일시: [날짜 및 시간]
장소: [회의 장소]
참석자: [참석자 목록]

회의에서는 [주요 안건]에 대해 논의할 예정이며, 각자 사전에 준비해 주시면 감사하겠습니다.

자세한 일정이나 문의 사항이 있으시면 언제든지 말씀해 주시기 바랍니다.

감사합니다.

[당신의 이름]  
[당신의 직책]  
[당신의 연락처]  
[회사 이름]  



이메일 제목: 다음주 월요일 프로텍트 회의 일정 공지

안녕하세요 팀 여러분,

다음 주 월요일에 열릴 프로텍트 회의 일정에 대해 안내드립니다.

일시: [날짜 및 시간]
장소: [회의 장소]
참석자: [참석자 목록]

회의에서는 [주요 안건]에 대해 논의할 예정이며, 각자 사전에 준비해 주시면 감사하겠습니다.

자세한 일정이나 문의 사항이 있으시면 언제든지 말씀해 주시기 바랍니다.

감사합니다.

[당신의


In [ ]:
# 모델 생성
